# Implementation of *[An embarrassingly simple approach to zero-shot learning](http://proceedings.mlr.press/v37/romera-paredes15.pdf)*

In [1]:
import numpy as np
from scipy.stats import hmean

from sklearn.metrics import accuracy_score

from scarce_shot_learn.data.awa2 import load_awa2
from scarce_shot_learn.zero import ESZSLearner

In [2]:
awa2_dataset = load_awa2()

In [3]:
X_train, attributes_features_train, labels_train = awa2_dataset['train']
X_val, attributes_features_val, labels_val = awa2_dataset['val']
X_test, attributes_features_test, labels_test = awa2_dataset['test']

In [4]:
attributes_features_train.shape

(27, 85)

In [5]:
labels_val

array([[10],
       [ 5],
       [11],
       ...,
       [12],
       [ 3],
       [11]], dtype=uint8)

### Classes 

In [6]:
train_classes = set(labels_train.reshape(-1))
val_classes = set(labels_val.reshape(-1))
test_classes = set(labels_test.reshape(-1))

print('No of training classes:', len(train_classes))
print('No of validation classes:', len(val_classes))
print('No of test classes:', len(test_classes))

No of training classes: 27
No of validation classes: 13
No of test classes: 10


## Model

Zero-shot learning models usually assume input data

$X$ - data features, a $num\_examples \times num\_dimensions$ matrix.

$S$ - data attribute features, a $num\_classes \times num\_attribute\_dimensions$ matrix.

$y$ - classes.

In [7]:
eszs_learner = ESZSLearner(lmbda=0.1, gamma=100)

In [8]:
help(ESZSLearner.fit)

Help on function fit in module scarce_shot_learn.zero.eszsl:

fit(self, X, y, class_attributes)
            X: {array-like, sparse matrix} of shape (n_samples, n_features)
    
            Training vector, where n_samples is the number of samples and n_features is the number of features.
    yarray-like of shape (n_samples,)
    
            y: array-like of shape (n_samples,)
            Target vector. n_classes = len(set(y))
    
            class_attributes: array-like of shape (training_n_classes, class_n_features)
            where training_n_classes >= n_classes
            classes from y map to attributes alphabetically



In [9]:
%%time

eszs_learner.fit(X_train, labels_train, attributes_features_train)

/usr/lib/python3/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


CPU times: user 31.8 s, sys: 30.6 s, total: 1min 2s
Wall time: 2.04 s


In [10]:
help(ESZSLearner.predict)

Help on function predict in module scarce_shot_learn.zero.eszsl:

predict(self, X, class_attributes, labels_to_attributes=None)
            X: {array-like, sparse matrix} of shape (n_samples, n_features)
    
            Feature vector, where n_samples is the number of samples and n_features is the number of features.
    yarray-like of shape (n_samples,)
    
            class_attributes: array-like of shape (prediction_n_classes, class_n_features)
    
            labels_to_attributes: array-like of shape (prediction_n_classes,)
            mapping between classes from class_attributes to prediction classes
            (classes that will be in y_pred)



In [11]:
predictions_train = eszs_learner.predict(X_train, attributes_features_train)

In [12]:
predictions_test = eszs_learner.predict(X_test, attributes_features_test)

In [13]:
train_accuracy = accuracy_score(labels_train, predictions_train)

In [14]:
test_accuracy = accuracy_score(labels_test, predictions_test)

### Final metric

Generalized Zero-Shot Learning uses harmonic mean of train and test accuracy as metric.

In [15]:
train_test_harmonic_mean = hmean([train_accuracy, test_accuracy])

In [16]:
print('train accuracy:', round(train_accuracy, 4))
print('test accuracy:', round(test_accuracy, 4))
print('train/test accuracy harmonic mean:', round(train_test_harmonic_mean, 4))

train accuracy: 0.9831
test accuracy: 0.4814
train/test accuracy harmonic mean: 0.6463
